In [3]:
import numpy as np
import xtrack as xt
import xobjects as xo
import xpart as xp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import constants 
from matplotlib.patches import Patch
#### PLOT THE DATA #######
SMALL_SIZE = 13.5
MEDIUM_SIZE = 17
BIGGER_SIZE = 23
plt.rcParams["font.family"] = "serif"
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


In [ ]:
#### Some notes:
'''
Angles and Offsets (of the H- beam) easy/safe to move. 'ELENABEAM/ECOOL-ANGLE-H-mrad'
E_k electrons: at a first order, only need to change  LNR4.ECVCATHLM/REF.DIRECT.V#VALUE
I electrons: need to act, at least, LNR4.ECVGRIDLM/REF.DIRECT.V#VALUE but will probably need to adjust energy as well
-> E_k and I_e are coupled! so you might need to change both at the same time...

The current of the electron, it is in principle measured by LNR4.ECVCOLLM/MEAS.I#VALUE

The current emitted (but not necessarily going into cooling) by the gun, is should be measurable by LNR4.ECVCATHLM/MEAS.I#VALUE

To change the delay of extraction:
AEX.MW-RF/Delay#delay
mind that with 10 ms (10 ticks) delay, you extract approximately at injection

To "not start" the e-cooling process, just set to False
AX.DK3-EC/OutEnable#outEnabled

To delay the start of cooling one can increase the value of
AX.DK3-EC/Delay#delay
Note: the default value=200 1kHz ticks means start at C time 9400, i.e. about 100 ms before our "second injection"


'''



############### Initial imports
# from pyjapcscout import PyJapcScout
from datascout import dict_to_parquet
from datascout import parquet_to_dict
from datascout import unixtime_to_string
import numpy as np
import datascout
import re
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter



data=np.load('H_angles/angles.npz')

n_steps=data['n_steps']
num_samples=data['num_samples']
delay_list=data['delay_list']
repeated_delay=data['repeated_delay']
angles_list=data['angles_list']
#horiztonal plane

angles_list=[-3,-2,-1,0,1,2,3]

angles_list = [0]

beta_x = 7.6 #m
beta_y = 1.3 #m

plt.figure(figsize=(12, 8))


for angle_values in angles_list:

    vertical_list = []
    horizontal_list = []

    vertical_integral_list = []
    horizontal_integral_list = []

    for i in range(len(repeated_delay)):
        print('index', i)
        data_to_look_at = parquet_to_dict(f'H_angles/angles{angle_values}/{i}.parquet')
        # BPM = data_to_look_at['LNE00.BSGW.0008/Acquisition']['value']
        BPM = data_to_look_at['LNE50.BSGW.5020/Acquisition']['value']
        vertical = BPM['sigma'][0]
        horizontal = BPM['sigma'][1]

        vertical_list.append(vertical)
        horizontal_list.append(horizontal)

        vertical_int = BPM['rawIntegrals'][0]
        horizontal_int = BPM['rawIntegrals'][1]

        vertical_integral_list.append(vertical_int)
        horizontal_integral_list.append(horizontal_int)

    filter_threshold_upper = 10
    filter_threshold_lower = 0.05
    integral_threshold = 5000

    filter_horizontal=[]
    filter_horizontal_intensity=[]
    filtered_delay_h=[]

    filter_vertical=[]
    filter_vertical_intensity=[]
    filtered_delay_v=[]

    sigma_x_list=[]

    for i in range(len(horizontal_list)):
        sigma_x=horizontal_list[i]
        sigma_y=vertical_list[i]
        integral_x=horizontal_integral_list[i]
        integral_y=vertical_integral_list[i]
        condition_sigma_x=filter_threshold_lower<sigma_x < filter_threshold_upper
        condition_sigma_y=filter_threshold_lower<sigma_y < filter_threshold_upper
        condition_intensity_x=integral_x > integral_threshold
        condition_intensity_y=integral_y > integral_threshold

        if condition_sigma_x & condition_sigma_y & condition_intensity_x & condition_intensity_y:
            filter_horizontal.append(sigma_x**2/beta_x)
            filter_vertical.append(sigma_y**2/beta_y)
            filter_horizontal_intensity.append(integral_x)
            filter_vertical_intensity.append(integral_y)
            filtered_delay_h.append(repeated_delay[i])
            filtered_delay_v.append(repeated_delay[i])

            
            

    diff_h=np.diff(filtered_delay_h)
    diff_v=np.diff(filtered_delay_v)

    indices_h = np.where(diff_h != 0)[0]+1
    indices_v = np.where(diff_v != 0)[0]+1

    sigma_h_groups = np.split(filter_horizontal,indices_h)
    sigma_v_groups = np.split(filter_vertical,indices_v)
    beamsize_h_groups = np.split(sigma_x_list,indices_h)

    h_delay_unique=np.unique(filtered_delay_h)
    v_delay_unique=np.unique(filtered_delay_v)

    means_h=[np.mean(group) for group in  sigma_h_groups]
    means_v=[np.mean(group) for group in sigma_v_groups]

    stds_h=[np.std(group) for group in sigma_h_groups]
    stds_v=[np.std(group) for group in sigma_v_groups]

    #means_sigma=[np.mean(group) for group in  beamsize_h_groups]

    np.savez(f'results/angle_x/horizontal/ELENA_angle{angle_values}.npz',
              h_delay_unique=h_delay_unique, means_h=means_h, stds_h=stds_h, angle_values=angle_values)
    np.savez(f'results/angle_x/vertical/ELENA_angle{angle_values}.npz',
              v_delay_unique=v_delay_unique, means_v=means_v, stds_v=stds_v, angle_values=angle_values)

    plt.errorbar(h_delay_unique,means_h,yerr=stds_h,label=f'angle={angle_values}')
    #plt.errorbar(v_delay_unique,means_v,yerr=stds_v,label=f'angle={angle_values}')
    plt.xlabel('Delay [ms]')
    plt.ylabel('$\epsilon_x$ [mm mrad]')
    plt.legend()
    plt.tight_layout()
plt.show()



<>:150: SyntaxWarning: invalid escape sequence '\e'
<>:150: SyntaxWarning: invalid escape sequence '\e'
/tmp/pkruyt/ipykernel_204148/1031407427.py:150: SyntaxWarning: invalid escape sequence '\e'
  plt.ylabel('$\epsilon_x$ [mm mrad]')


index 0


/tmp/pkruyt/ipykernel_204148/1031407427.py:150: SyntaxWarning: invalid escape sequence '\e'
  plt.ylabel('$\epsilon_x$ [mm mrad]')


KeyError: 'LNE50.BSGW.5020/Acquisition'

<Figure size 1200x800 with 0 Axes>

In [ ]:
means_v

[2.1372310895028126,
 2.402644718678169,
 1.8162499922986532,
 1.5287596785205564,
 1.1597636170725445,
 0.9492665103448836,
 0.7657572815373527,
 0.6777949433260902,
 0.6284386407816985,
 0.615493824883545]

In [ ]:
# Example usage:
k_b = 1.38e-23  # Replace with actual Boltzmann constant value
T_L = 300       # Replace with actual temperature in Kelvin
m_i = 1.67e-27  # Replace with actual mass in kilograms
beta_x_y = 0.1  # Replace with actual beta_x_y value
beta_c = 0.2    # Replace with actual beta_c value
x = 0.01        # Replace with actual x value
y = 0.02        # Replace with actual y value
